In [1]:
import os
import shutil
import numpy as np
import cv2
from sklearn.model_selection import train_test_split

In [8]:
dataset_path = "BUSI"

train_path = os.path.join(dataset_path, "train")
test_path = os.path.join(dataset_path, "test")

if not os.path.exists(train_path):
    os.makedirs(train_path)
if not os.path.exists(test_path):
    os.makedirs(test_path)

# make mask and image folders
train_mask_path = os.path.join(train_path, "mask")
train_image_path = os.path.join(train_path, "image")
test_mask_path = os.path.join(test_path, "mask")
test_image_path = os.path.join(test_path, "image")

if not os.path.exists(train_mask_path):
    os.makedirs(train_mask_path)
if not os.path.exists(train_image_path):
    os.makedirs(train_image_path)
if not os.path.exists(test_mask_path):
    os.makedirs(test_mask_path)
if not os.path.exists(test_image_path):
    os.makedirs(test_image_path)

In [9]:
images = []
masks = []
labels = []

for target in sorted(os.listdir(dataset_path)):
    d = os.path.join(dataset_path, target)
    if not os.path.isdir(d):
        continue

    for root, _, fnames in sorted(os.walk(d)):
        for fname in sorted(fnames):
            path = os.path.join(root, fname)

            if not "mask" in path:
                mask_path = path.replace('.png', '_mask.png')

                add_masks = []
                for i in range(1, 5):
                    mask_path_add = mask_path.replace('mask', 'mask_' + str(i))
                    if os.path.isfile(mask_path_add):
                        add_masks.append(mask_path_add)


                if os.path.isfile(mask_path):
                    images.append(path)
                    if len(add_masks) > 0:
                        masks.append(add_masks + [mask_path])
                    else:
                        masks.append([mask_path])
                    labels.append(target)

In [10]:
print("Total images: ", len(images))

indices = np.arange(len(images))

# split images
train_indices, test_indices = train_test_split(indices, test_size=0.2, random_state=42)

Total images:  780


In [11]:
def add_masks(mask_paths):
    mask = cv2.imread(mask_paths[0], cv2.IMREAD_GRAYSCALE)
    for i in range(1, len(mask_paths)):
        mask_add = cv2.imread(mask_paths[i], cv2.IMREAD_GRAYSCALE)
        mask = cv2.add(mask, mask_add)
    return mask

In [12]:
for i in train_indices:
    shutil.copy(images[i], train_image_path)
    # shutil.copy(masks[i], train_mask_path)

    masks_added = add_masks(masks[i])
    mask_name = masks[i][-1].split('/')[-1]
    new_mask_name = mask_name.replace('_mask', '')
    new_mask_path = os.path.join(train_mask_path, new_mask_name)
    cv2.imwrite(new_mask_path, masks_added)

for i in test_indices:
    shutil.copy(images[i], test_image_path)
    # shutil.copy(masks[i], test_mask_path)
    
    masks_added = add_masks(masks[i])
    mask_name = masks[i][-1].split('/')[-1]
    new_mask_name = mask_name.replace('_mask', '')
    new_mask_path = os.path.join(test_mask_path, new_mask_name)
    cv2.imwrite(new_mask_path, masks_added)

In [9]:
name = "benign_(132).png"

print("mask" in name)

False
